In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
from utils import cloud_constants as cc
from utils import text_normalizer as tn
from utils import bert_text_processor as btp
from models import bert_cve_classifier as bcvec
from models import security_dl_classifier as sdc
from models import cve_dl_classifier as cdc
from collections import Counter


import pandas as pd
import numpy as np
import arrow
import daiquiri
import logging
import gc
import os
import argparse
import textwrap
import tensorflow as tf

I0811 07:46:46.257797 140292808554304 __init__.py:56] hub.KerasLayer is not available because TensorFlow version is less than 1.14
Using TensorFlow backend.


In [16]:
df = pd.read_csv('./test_data_models.csv', encoding='utf-8', na_filter=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4674 entries, 0 to 4673
Data columns (total 15 columns):
repo_name       4674 non-null object
event_type      4674 non-null object
ecosystem       4674 non-null object
status          4674 non-null object
id              4674 non-null int64
number          4674 non-null int64
api_url         4674 non-null object
url             4674 non-null object
creator_name    4674 non-null object
creator_url     4674 non-null object
created_at      4674 non-null object
updated_at      4674 non-null object
closed_at       4674 non-null object
title           4674 non-null object
body            4674 non-null object
dtypes: int64(2), object(13)
memory usage: 547.8+ KB


In [17]:
df.head()

,repo_name,event_type,ecosystem,status,id,number,api_url,url,creator_name,creator_url,created_at,updated_at,closed_at,title,body
0,AlecAivazis/survey,IssuesEvent,golang,closed,476207931,241,https://api.github.com/repos/AlecAivazis/surve...,https://github.com/AlecAivazis/survey/issues/241,mkumatag,https://github.com/mkumatag,2019-08-02 14:28:10+00:00,2019-08-02 15:38:51+00:00,2019-08-02 15:38:51+00:00,v2 version from the code should be removed,The specific version(e.g: github.com/AlecAivaz...
1,Azure/azure-sdk-for-go,IssuesEvent,golang,opened,474328235,5379,https://api.github.com/repos/Azure/azure-sdk-f...,https://github.com/Azure/azure-sdk-for-go/issu...,mguglani,https://github.com/mguglani,2019-07-30 01:37:13+00:00,2019-07-30 01:37:13+00:00,,MarshalJSON for EvenData return nil,### Bug Report - What happened? I am using Res...
2,Azure/azure-sdk-for-go,IssuesEvent,golang,opened,475079134,5391,https://api.github.com/repos/Azure/azure-sdk-f...,https://github.com/Azure/azure-sdk-for-go/issu...,invidian,https://github.com/invidian,2019-07-31 11:05:29+00:00,2019-07-31 11:05:29+00:00,,Support for client-side blob encryption,### Feature Request It seems that Blob Encrypt...
3,Azure/azure-sdk-for-go,IssuesEvent,golang,opened,475785847,5407,https://api.github.com/repos/Azure/azure-sdk-f...,https://github.com/Azure/azure-sdk-for-go/issu...,mbfrahry,https://github.com/mbfrahry,2019-08-01 16:47:37+00:00,2019-08-01 16:47:37+00:00,,App Service Plan 404 does not return an error,### Bug Report - import path of package in que...
4,Azure/azure-sdk-for-go,IssuesEvent,golang,opened,475914536,5412,https://api.github.com/repos/Azure/azure-sdk-f...,https://github.com/Azure/azure-sdk-for-go/issu...,udbhav-sharma,https://github.com/udbhav-sharma,2019-08-01 22:23:26+00:00,2019-08-01 22:23:26+00:00,,Authenticating with Azure using Certificate,I am trying to authenticate with Azure AAD usi...


In [18]:
daiquiri.setup(level=logging.INFO)
_logger = daiquiri.getLogger(__name__)

In [19]:
_logger.info('Creating description column for NLP')
df['description'] = df['title'].fillna(value='').map(
    str) + ' ' + df['body'].fillna(value='')
columns = ['title', 'body']
df.drop(columns, inplace=True, axis=1)

2019-08-11 10:14:39,681 [7770] INFO     __main__: Creating description column for NLP


In [20]:
_logger.info('----- STARTING MODEL INFERENCE -----')
_logger.info('Loading Security Model')
sc = sdc.SecurityClassifier(embedding_size=300, max_length=1000, max_features=800000,
                            tokenizer_path=cc.P1GRU_SEC_MODEL_TOKENIZER_PATH,
                            model_weights_path=cc.P1GRU_SEC_MODEL_WEIGHTS_PATH)
sc.build_model_architecture()
sc.load_model_weights()
sc_model = sc.get_model()
sc_model.summary()

2019-08-11 10:14:41,047 [7770] INFO     __main__: ----- STARTING MODEL INFERENCE -----
2019-08-11 10:14:41,049 [7770] INFO     __main__: Loading Security Model
2019-08-11 10:14:41,050 [7770] INFO     models.security_dl_classifier: Loading Security Model Tokenizer Vocabulary
2019-08-11 10:14:41,332 [7770] INFO     models.security_dl_classifier: Building Security Model Architecture
2019-08-11 10:14:42,005 [7770] INFO     models.security_dl_classifier: Loading Security Model Weights


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 1000, 300)         194814900 
_________________________________________________________________
bidirectional_3 (Bidirection (None, 1000, 128)         140544    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 1000, 64)          31104     
_________________________________________________________________
attention_layer_2 (Attention (None, 64)                1064      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
__________

In [21]:
_logger.info('Text Pre-processing Issue/PR Descriptions')
df['norm_description'] = tn.pre_process_documents_parallel(
    documents=df['description'].values)

_logger.info('Setting Default CVE and Security Flags')
df['security_model_flag'] = 0
df['cve_model_flag'] = 0

_logger.info('\n')

2019-08-11 10:15:47,915 [7770] INFO     __main__: Text Pre-processing Issue/PR Descriptions
2019-08-11 10:15:47,919 [7770] INFO     utils.text_normalizer: Text Pre-processing: starting
2019-08-11 10:15:47,921 [7770] INFO     utils.text_normalizer: ThreadPoolExecutor-1_0: working on doc num: 0
2019-08-11 10:16:03,203 [7770] INFO     utils.text_normalizer: ThreadPoolExecutor-1_9: working on doc num: 4673
2019-08-11 10:16:03,284 [7770] INFO     __main__: Setting Default CVE and Security Flags
2019-08-11 10:16:03,286 [7770] INFO     __main__: 



In [22]:
_logger.info('Preparing data for security model inference')
security_encoded_docs = sc.prepare_inference_data(
    df['norm_description'].tolist())
_logger.info('Total Security Docs Encoded: {n}'.format(
    n=len(security_encoded_docs)))
sec_doc_lengths = np.array([len(np.nonzero(item)[0])
                            for item in security_encoded_docs])
_logger.info('Removing bad docs with low tokens')
sec_doc_idx = np.argwhere(sec_doc_lengths >= 5).ravel()
filtered_security_encoded_docs = security_encoded_docs[sec_doc_idx]
_logger.info('Filtered Security Docs Encoded: {n}'.format(
    n=len(filtered_security_encoded_docs)))

_logger.info('Making predictions for probable security issues')
sec_pred_probs = sc_model.predict(
    filtered_security_encoded_docs, batch_size=1024, verbose=0)
sec_pred_probsr = sec_pred_probs.ravel()
sec_pred_labels = [1 if prob > 0.4 else 0 for prob in sec_pred_probsr]
_logger.info('Updating Security Model predictions in dataset')
df.loc[df.index.isin(sec_doc_idx), 'security_model_flag'] = sec_pred_labels

_logger.info('Teardown security model')
del sc
del sc_model
gc.collect()

_logger.info('\n')

2019-08-11 10:16:08,594 [7770] INFO     __main__: Preparing data for security model inference
2019-08-11 10:16:08,944 [7770] INFO     __main__: Total Security Docs Encoded: 4674
2019-08-11 10:16:08,972 [7770] INFO     __main__: Removing bad docs with low tokens
2019-08-11 10:16:08,984 [7770] INFO     __main__: Filtered Security Docs Encoded: 4575
2019-08-11 10:16:08,985 [7770] INFO     __main__: Making predictions for probable security issues
2019-08-11 10:16:17,591 [7770] INFO     __main__: Updating Security Model predictions in dataset
2019-08-11 10:16:17,597 [7770] INFO     __main__: Teardown security model
2019-08-11 10:16:17,820 [7770] INFO     __main__: 



In [23]:
subset_df = df[df['security_model_flag'] == 1]
subset_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 713 entries, 1 to 4653
Data columns (total 17 columns):
repo_name              713 non-null object
event_type             713 non-null object
ecosystem              713 non-null object
status                 713 non-null object
id                     713 non-null int64
number                 713 non-null int64
api_url                713 non-null object
url                    713 non-null object
creator_name           713 non-null object
creator_url            713 non-null object
created_at             713 non-null object
updated_at             713 non-null object
closed_at              713 non-null object
description            713 non-null object
norm_description       713 non-null object
security_model_flag    713 non-null int64
cve_model_flag         713 non-null int64
dtypes: int64(4), object(13)
memory usage: 100.3+ KB


In [24]:
subset_df.head()

,repo_name,event_type,ecosystem,status,id,number,api_url,url,creator_name,creator_url,created_at,updated_at,closed_at,description,norm_description,security_model_flag,cve_model_flag
1,Azure/azure-sdk-for-go,IssuesEvent,golang,opened,474328235,5379,https://api.github.com/repos/Azure/azure-sdk-f...,https://github.com/Azure/azure-sdk-for-go/issu...,mguglani,https://github.com/mguglani,2019-07-30 01:37:13+00:00,2019-07-30 01:37:13+00:00,,MarshalJSON for EvenData return nil ### Bug Re...,marshaljson for evendata return nil bug report...,1,0
4,Azure/azure-sdk-for-go,IssuesEvent,golang,opened,475914536,5412,https://api.github.com/repos/Azure/azure-sdk-f...,https://github.com/Azure/azure-sdk-for-go/issu...,udbhav-sharma,https://github.com/udbhav-sharma,2019-08-01 22:23:26+00:00,2019-08-01 22:23:26+00:00,,Authenticating with Azure using Certificate I ...,authenticating with azure using certificate i ...,1,0
6,Azure/azure-sdk-for-go,IssuesEvent,golang,opened,476478191,5425,https://api.github.com/repos/Azure/azure-sdk-f...,https://github.com/Azure/azure-sdk-for-go/issu...,djyou,https://github.com/djyou,2019-08-03 18:38:18+00:00,2019-08-03 18:38:18+00:00,,Table query not extracting and following conti...,table query not extracting and following conti...,1,0
7,Azure/go-autorest,IssuesEvent,golang,closed,460244874,413,https://api.github.com/repos/Azure/go-autorest...,https://github.com/Azure/go-autorest/issues/413,justaugustus,https://github.com/justaugustus,2019-06-25 06:53:55+00:00,2019-08-01 15:24:36+00:00,2019-08-01 15:24:36+00:00,Cannot bump dependencies in Kubernetes project...,cannot bump dependencies in kubernetes project...,1,0
33,aws/aws-sdk-go,IssuesEvent,golang,closed,307159642,1853,https://api.github.com/repos/aws/aws-sdk-go/is...,https://github.com/aws/aws-sdk-go/issues/1853,svenwltr,https://github.com/svenwltr,2018-03-21 08:51:17+00:00,2019-08-02 19:17:06+00:00,2019-08-02 19:17:05+00:00,Unable to delete Object in empty dir with s3.D...,unable to delete object in empty dir with s3 d...,1,0


In [25]:
subset_df.to_csv('sec_issues_aug01.csv')

In [26]:
sess = tf.Session()
BERT_MAX_SEQ_LEN = 512

_logger.info('Loading CVE Model')
bc = bcvec.BERTClassifier(bert_model_path=cc.BASE_BERT_UNCASED_PATH, 
                  max_seq_length=BERT_MAX_SEQ_LEN)
bc.build_model_architecture()   

subset_df['norm_description'] = tn.pre_process_documents_parallel_bert(
                                    documents=subset_df['description'].values) 
cve_encoded_docs = subset_df['norm_description'].values
_logger.info('Total CVE Docs Encoded: {n}'.format(
    n=len(cve_encoded_docs)))
cve_doc_lengths = np.array([len(doc.split(' ')) for doc in cve_encoded_docs])
_logger.info('Removing bad docs with low tokens')
cve_doc_idx = np.argwhere(cve_doc_lengths >= 10).ravel()
filtered_cve_encoded_docs = cve_encoded_docs[cve_doc_idx]
_logger.info('Filtered CVE Docs Encoded: {n}'.format(
    n=len(filtered_cve_encoded_docs)))

_logger.info('BERT text processing and feature engineering')
btp_obj = btp.BertTextProcessor(tf_session=sess, 
                          bert_model_path=cc.BASE_BERT_UNCASED_PATH, 
                          max_seq_length=BERT_MAX_SEQ_LEN)
btp_obj.create_bert_tokenizer()
btp_obj.convert_text_to_input_examples(filtered_cve_encoded_docs)
btp_obj.convert_examples_to_features()

2019-08-11 10:16:35,737 [7770] INFO     __main__: Loading CVE Model


Build BERT Classifier CVE Model Architecture
Loading Base BERT Model
Trainable layers: 199
Non Trainable layers: 5
Constructing Base BERT architecture


2019-08-11 10:16:38,740 [7770] INFO     tensorflow: Saver not created because there are no variables in the graph to restore
2019-08-11 10:16:38,963 [7770] INFO     utils.text_normalizer: Text Pre-processing: starting
2019-08-11 10:16:38,964 [7770] INFO     utils.text_normalizer: ThreadPoolExecutor-2_0: working on doc num: 0
2019-08-11 10:16:39,882 [7770] INFO     utils.text_normalizer: ThreadPoolExecutor-2_3: working on doc num: 712
2019-08-11 10:16:40,302 [7770] INFO     __main__: Total CVE Docs Encoded: 713
2019-08-11 10:16:40,314 [7770] INFO     __main__: Removing bad docs with low tokens
2019-08-11 10:16:40,315 [7770] INFO     __main__: Filtered CVE Docs Encoded: 692
2019-08-11 10:16:40,316 [7770] INFO     __main__: BERT text processing and feature engineering


Loading Base BERT Model


2019-08-11 10:16:43,881 [7770] INFO     tensorflow: Saver not created because there are no variables in the graph to restore
Converting text to examples: 692it [00:00, 262761.03it/s]
Converting examples to features:   0%|          | 0/692 [00:00<?, ?it/s]

Loading BERT WordPiece Tokenizer
Creating Input Examples from data
Creating BERT Input Features from Input Examples


Converting examples to features: 100%|██████████| 692/692 [00:05<00:00, 131.34it/s]


In [27]:
prob_security_df_rowidx = np.array(subset_df.index)

In [67]:
_logger.info('Making predictions for probable CVE issues')
btp.initialize_vars(sess)
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve75iter2_weights-ep:01-trn_loss:0.379-trn_acc:0.904-val_loss:0.229-val_acc:0.918.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

2019-08-11 12:03:00,513 [7770] INFO     __main__: Making predictions for probable CVE issues


Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 32s 46ms/step
Counter({0: 610, 1: 82})
(array([  2,   9,  11,  36,  84,  91, 124, 125, 128, 130, 147, 150, 151,
       154, 163, 172, 175, 178, 179, 180, 211, 213, 242, 252, 256, 257,
       262, 284, 290, 293, 312, 324, 334, 343, 355, 389, 417, 421, 428,
       455, 457, 462, 488, 491, 494, 496, 498, 500, 511, 527, 541, 552,
       564, 565, 570, 578, 580, 593, 598, 622, 637, 639, 643, 650, 651,
       654, 657, 658, 659, 660, 661, 662, 665, 666, 670, 671, 672, 676,
       684, 685, 687, 688]),)


In [68]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve75iter2_weights-ep:02-trn_loss:0.176-trn_acc:0.959-val_loss:0.153-val_acc:0.941.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 32s 47ms/step
Counter({0: 562, 1: 130})
(array([  2,   9,  11,  64,  84,  92, 129, 130, 150, 153, 154, 159, 162,
       165, 166, 168, 169, 170, 172, 173, 178, 180, 181, 193, 199, 213,
       226, 227, 229, 245, 252, 262, 268, 269, 278, 284, 286, 290, 291,
       299, 311, 312, 315, 328, 336, 338, 349, 351, 353, 361, 367, 370,
       389, 417, 422, 424, 432, 433, 445, 450, 455, 457, 459, 462, 463,
       465, 469, 477, 479, 482, 488, 496, 498, 511, 519, 521, 525, 527,
       540, 541, 544, 547, 550, 570, 572, 577, 578, 580, 581, 582, 588,
       589, 593, 594, 597, 598, 599, 601, 605, 621, 637, 639, 643, 645,
       650, 651, 654, 657, 658, 659, 660, 661, 662, 664, 665, 666, 670,
       671, 672, 673, 674, 676, 679, 682, 684, 685, 686, 687, 688, 691]),)


In [69]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve75iter2_weights-ep:03-trn_loss:0.096-trn_acc:0.981-val_loss:0.284-val_acc:0.976.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 33s 48ms/step
Counter({0: 662, 1: 30})
(array([  9, 130, 147, 150, 178, 180, 199, 213, 226, 242, 262, 268, 315,
       328, 349, 389, 417, 432, 433, 450, 455, 457, 491, 622, 639, 643,
       650, 671, 673, 685]),)


In [70]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve75iter2_weights-ep:04-trn_loss:0.053-trn_acc:0.987-val_loss:0.265-val_acc:0.976.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 34s 49ms/step
Counter({0: 669, 1: 23})
(array([ 92, 130, 150, 161, 178, 180, 213, 268, 432, 433, 450, 455, 457,
       491, 496, 637, 643, 650, 661, 671, 676, 679, 691]),)


In [71]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve75iter2_weights-ep:05-trn_loss:0.055-trn_acc:0.991-val_loss:0.233-val_acc:0.956.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 34s 49ms/step
Counter({0: 644, 1: 48})
(array([  2,   9,  92, 130, 150, 154, 161, 165, 172, 178, 199, 213, 257,
       268, 290, 315, 328, 349, 415, 432, 433, 450, 455, 457, 469, 488,
       491, 496, 498, 561, 580, 593, 601, 613, 621, 622, 637, 639, 643,
       650, 657, 671, 673, 676, 684, 685, 687, 691]),)


In [72]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve75iter2_weights-ep:06-trn_loss:0.044-trn_acc:0.994-val_loss:0.239-val_acc:0.931.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 34s 49ms/step
Counter({0: 565, 1: 127})
(array([  2,   9,  11,  12,  18,  19,  41,  84,  88,  91,  92,  93, 119,
       128, 130, 137, 139, 141, 142, 147, 149, 150, 151, 154, 156, 157,
       161, 165, 166, 169, 170, 172, 178, 180, 181, 191, 192, 198, 199,
       203, 209, 211, 212, 213, 214, 227, 229, 230, 235, 242, 244, 245,
       246, 248, 249, 257, 258, 260, 264, 268, 269, 274, 277, 279, 284,
       286, 290, 293, 314, 315, 328, 334, 349, 350, 351, 361, 369, 371,
       385, 389, 422, 432, 433, 436, 440, 450, 455, 457, 467, 469, 474,
       488, 491, 496, 498, 499, 511, 522, 527, 561, 577, 580, 588, 594,
       597, 598, 599, 601, 606, 611, 621, 622, 637, 639, 643, 650, 656,
       657, 663, 671, 673, 676, 679, 684, 685, 687, 688]),)


In [73]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve75iter2_weights-ep:07-trn_loss:0.062-trn_acc:0.990-val_loss:0.303-val_acc:0.985.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 34s 50ms/step
Counter({0: 682, 1: 10})
(array([226, 268, 432, 450, 455, 637, 643, 650, 671, 685]),)


In [74]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve75iter2_weights-ep:08-trn_loss:0.021-trn_acc:0.998-val_loss:0.377-val_acc:0.986.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 50ms/step
Counter({0: 684, 1: 8})
(array([268, 432, 450, 455, 639, 643, 671, 685]),)


In [75]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve75iter2_weights-ep:09-trn_loss:0.024-trn_acc:0.998-val_loss:0.330-val_acc:0.971.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 50ms/step
Counter({0: 671, 1: 21})
(array([  9,  12,  92, 130, 161, 213, 257, 268, 450, 455, 457, 494, 588,
       622, 639, 643, 650, 671, 676, 684, 685]),)


In [76]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve75iter2_weights-ep:10-trn_loss:0.039-trn_acc:0.994-val_loss:0.239-val_acc:0.972.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 50ms/step
Counter({0: 661, 1: 31})
(array([  9,  64, 178, 181, 268, 275, 351, 424, 432, 450, 455, 457, 469,
       488, 491, 496, 588, 594, 601, 622, 637, 639, 640, 643, 650, 657,
       671, 676, 682, 685, 687]),)


In [77]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve75_weights-ep:01-trn_loss:0.395-trn_acc:0.888-val_loss:0.188-val_acc:0.938.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 51ms/step
Counter({0: 585, 1: 107})
(array([  9,  11,  84,  92, 128, 130, 150, 153, 154, 157, 159, 161, 162,
       165, 166, 167, 168, 169, 170, 172, 173, 178, 180, 181, 193, 216,
       242, 252, 268, 284, 330, 351, 355, 372, 384, 417, 433, 455, 457,
       458, 459, 460, 461, 462, 465, 466, 479, 482, 488, 496, 498, 511,
       516, 518, 521, 522, 524, 530, 535, 536, 537, 540, 541, 544, 548,
       550, 553, 554, 560, 561, 563, 564, 565, 567, 570, 572, 573, 576,
       577, 578, 580, 581, 582, 586, 587, 588, 589, 591, 593, 596, 598,
       637, 639, 643, 650, 657, 658, 661, 663, 671, 674, 681, 682, 685,
       687, 688, 691]),)


In [78]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve75_weights-ep:02-trn_loss:0.172-trn_acc:0.957-val_loss:0.164-val_acc:0.978.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 51ms/step
Counter({0: 673, 1: 19})
(array([  9, 130, 132, 268, 312, 315, 328, 349, 351, 389, 417, 432, 433,
       455, 611, 622, 639, 671, 685]),)


In [79]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve75_weights-ep:03-trn_loss:0.106-trn_acc:0.971-val_loss:0.173-val_acc:0.963.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 51ms/step
Counter({0: 646, 1: 46})
(array([  2,   9,  52,  84, 130, 161, 166, 214, 238, 262, 268, 284, 351,
       355, 389, 417, 432, 433, 455, 457, 461, 462, 465, 467, 496, 519,
       541, 570, 572, 588, 593, 597, 599, 601, 611, 637, 639, 643, 650,
       661, 671, 676, 681, 682, 687, 688]),)


In [80]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve75_weights-ep:04-trn_loss:0.053-trn_acc:0.988-val_loss:0.244-val_acc:0.982.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 51ms/step
Counter({0: 671, 1: 21})
(array([  9, 128, 130, 144, 166, 268, 328, 351, 389, 432, 455, 465, 570,
       588, 622, 639, 643, 650, 661, 671, 676]),)


In [81]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve75_weights-ep:05-trn_loss:0.052-trn_acc:0.990-val_loss:0.288-val_acc:0.983.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 51ms/step
Counter({0: 685, 1: 7})
(array([  9, 130, 433, 455, 639, 671, 685]),)


In [82]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve99_weights-ep:01-trn_loss:0.375-trn_acc:0.906-val_loss:0.278-val_acc:0.984.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 51ms/step
Counter({0: 679, 1: 13})
(array([  9, 144, 180, 268, 433, 622, 639, 643, 650, 661, 663, 671, 685]),)


In [83]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve99_weights-ep:02-trn_loss:0.194-trn_acc:0.953-val_loss:0.217-val_acc:0.948.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 51ms/step
Counter({0: 621, 1: 71})
(array([  2,   9,  11,  26,  92,  93, 130, 144, 154, 161, 169, 178, 180,
       199, 202, 203, 210, 226, 228, 235, 241, 257, 264, 268, 269, 277,
       279, 284, 289, 314, 334, 361, 422, 433, 440, 450, 455, 457, 469,
       476, 488, 490, 491, 494, 495, 496, 497, 511, 519, 541, 561, 577,
       578, 580, 588, 593, 597, 601, 606, 621, 622, 637, 639, 643, 650,
       656, 657, 661, 663, 671, 676]),)


In [84]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve99_weights-ep:03-trn_loss:0.111-trn_acc:0.976-val_loss:0.308-val_acc:0.980.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 51ms/step
Counter({0: 679, 1: 13})
(array([144, 268, 433, 455, 519, 622, 639, 643, 650, 657, 663, 671, 685]),)


In [85]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve99_weights-ep:04-trn_loss:0.075-trn_acc:0.984-val_loss:0.275-val_acc:0.983.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 51ms/step
Counter({0: 677, 1: 15})
(array([  9, 154, 178, 433, 455, 511, 519, 622, 637, 639, 643, 650, 671,
       676, 682]),)


In [86]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve99_weights-ep:05-trn_loss:0.076-trn_acc:0.983-val_loss:0.368-val_acc:0.981.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 51ms/step
Counter({0: 682, 1: 10})
(array([  9, 269, 433, 455, 457, 611, 639, 643, 671, 688]),)


In [87]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve99iter2_weights-ep:01-trn_loss:0.380-trn_acc:0.915-val_loss:0.229-val_acc:0.927.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 51ms/step
Counter({0: 582, 1: 110})
(array([  2,   9,  11,  26,  36,  84,  92, 130, 150, 153, 154, 155, 159,
       165, 172, 173, 178, 179, 180, 181, 213, 220, 227, 228, 268, 269,
       284, 286, 289, 312, 319, 336, 351, 352, 355, 361, 383, 384, 389,
       417, 424, 433, 440, 450, 455, 457, 461, 462, 465, 466, 468, 469,
       477, 480, 484, 488, 496, 498, 511, 512, 519, 520, 523, 537, 541,
       544, 547, 548, 561, 563, 564, 565, 567, 570, 572, 577, 578, 580,
       581, 588, 593, 594, 597, 599, 601, 622, 637, 639, 643, 645, 650,
       651, 654, 657, 658, 659, 660, 661, 662, 665, 666, 670, 671, 672,
       676, 679, 681, 682, 688, 691]),)


In [88]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve99iter2_weights-ep:02-trn_loss:0.166-trn_acc:0.963-val_loss:0.200-val_acc:0.947.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 51ms/step
Counter({0: 619, 1: 73})
(array([  9,  11,  26,  92,  93, 130, 144, 147, 150, 154, 155, 178, 180,
       199, 202, 211, 212, 213, 227, 228, 230, 235, 238, 241, 242, 244,
       257, 262, 264, 268, 269, 277, 289, 319, 324, 334, 349, 352, 389,
       417, 432, 433, 440, 450, 455, 457, 476, 488, 496, 506, 534, 580,
       588, 594, 597, 599, 601, 621, 622, 637, 639, 643, 650, 661, 663,
       671, 673, 676, 681, 684, 685, 687, 688]),)


In [89]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve99iter2_weights-ep:03-trn_loss:0.098-trn_acc:0.978-val_loss:0.255-val_acc:0.981.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 51ms/step
Counter({0: 677, 1: 15})
(array([150, 180, 228, 268, 277, 289, 334, 389, 433, 588, 639, 643, 650,
       671, 676]),)


In [90]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve99iter2_weights-ep:04-trn_loss:0.058-trn_acc:0.990-val_loss:0.232-val_acc:0.983.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 51ms/step
Counter({0: 678, 1: 14})
(array([150, 269, 389, 433, 450, 457, 588, 601, 637, 639, 643, 650, 671,
       676]),)


In [91]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve99iter2_weights-ep:05-trn_loss:0.040-trn_acc:0.994-val_loss:0.306-val_acc:0.984.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 50ms/step
Counter({0: 683, 1: 9})
(array([150, 269, 450, 588, 637, 639, 643, 671, 676]),)


In [92]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve99iter2_weights-ep:06-trn_loss:0.026-trn_acc:0.994-val_loss:0.219-val_acc:0.964.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 51ms/step
Counter({0: 648, 1: 44})
(array([  2,   9,  11,  92, 130, 150, 154, 161, 178, 199, 209, 212, 228,
       264, 269, 271, 277, 279, 289, 334, 389, 433, 440, 450, 455, 457,
       488, 496, 570, 588, 593, 597, 599, 601, 604, 622, 637, 639, 643,
       650, 658, 671, 676, 679]),)


In [93]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve99iter2_weights-ep:07-trn_loss:0.025-trn_acc:0.994-val_loss:0.286-val_acc:0.970.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 51ms/step
Counter({0: 652, 1: 40})
(array([  9, 130, 144, 150, 154, 178, 262, 269, 319, 334, 352, 389, 450,
       455, 457, 463, 506, 519, 541, 547, 561, 570, 588, 599, 601, 637,
       639, 643, 650, 656, 663, 669, 670, 671, 676, 677, 680, 684, 685,
       686]),)


In [94]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve99iter2_weights-ep:08-trn_loss:0.005-trn_acc:0.998-val_loss:1.037-val_acc:0.983.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 51ms/step
Counter({0: 688, 1: 4})
(array([389, 650, 671, 676]),)


In [95]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve99iter2_weights-ep:09-trn_loss:0.029-trn_acc:0.995-val_loss:0.404-val_acc:0.982.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 50ms/step
Counter({0: 677, 1: 15})
(array([130, 150, 295, 334, 455, 588, 603, 637, 639, 650, 663, 671, 676,
       677, 685]),)


In [96]:
MODEL_PATH = '../../../dsarkar/saved_models/bert_vuln_models/bert_cve99iter2_weights-ep:10-trn_loss:0.025-trn_acc:0.996-val_loss:0.454-val_acc:0.982.h5'
bc.load_model_weights(model_weights_path=MODEL_PATH)

cve_pred_probs = bc.model_estimator.predict(x=[btp_obj.input_ids, 
                                               btp_obj.input_masks, 
                                               btp_obj.segment_ids],
                                            batch_size=200,
                                            verbose=1)
cve_pred_probsr = cve_pred_probs.ravel()
cve_pred_labels = [1 if prob > 0.5 else 0 for prob in cve_pred_probsr]
print(Counter(cve_pred_labels))
print(np.nonzero(cve_pred_labels))

Loading BERT Classifier CVE Model Weights
692/692 [==============================] - 35s 51ms/step
Counter({0: 681, 1: 11})
(array([150, 269, 295, 455, 637, 643, 650, 663, 671, 685, 688]),)


In [114]:
for i, v in enumerate(subset_df.iloc[ [150, 269, 389, 433, 450, 457, 588, 601, 637, 639, 643, 650, 671,
       676]]['url'].values):
    print(i, ':->', v)
    
    
     

0 :-> https://github.com/influxdata/influxdb/issues/10542
1 :-> https://github.com/influxdata/influxdb/issues/9468
2 :-> https://github.com/kubernetes/kubernetes/issues/78467
3 :-> https://github.com/openshift/openshift-ansible/issues/11781
4 :-> https://github.com/operator-framework/operator-sdk/issues/1771
5 :-> https://github.com/sirupsen/logrus/issues/999
6 :-> https://github.com/istio/istio/pull/16047
7 :-> https://github.com/knative/serving/pull/4958
8 :-> https://github.com/kubernetes/kubernetes/pull/80730
9 :-> https://github.com/kubernetes/kubernetes/pull/80764
10 :-> https://github.com/kubernetes/kubernetes/pull/80905
11 :-> https://github.com/kubernetes/test-infra/pull/13702
12 :-> https://github.com/openshift/console/pull/2193
13 :-> https://github.com/openshift/image-registry/pull/184


In [112]:
subset_df.iloc[389]['url']

'https://github.com/kubernetes/kubernetes/issues/78467'